In [ ]:
pip install fastcluster

In [ ]:
import os
import sys
import time
# import hdbscan
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import IPython.display as ipd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
# import missingno as mno
from sklearn.neighbors import KNeighborsRegressor, kneighbors_graph, NearestNeighbors
# from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer #, IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
# from sklearn.neural_network import MLPRegressor
from sklearn.cluster import AgglomerativeClustering, KMeans, Birch
from sklearn.semi_supervised import LabelPropagation
from sklearn.datasets import make_blobs
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist, squareform, euclidean
from scipy.sparse import lil_matrix
from fastcluster import linkage as flinkage
from collections import Counter

In [ ]:
data = pd.read_csv("/kaggle/input/private-blood-count/labelled_second_all_dropped.csv")
data

In [ ]:
data['Diagnosis'] = data['Diagnosis'].str.replace('Chronic myeloid leukemia in chronic phase', 'Chronic myeloid leukemia chronic phase')

In [ ]:
first = data.iloc[:, :58]

In [ ]:
first

In [ ]:
scaler = StandardScaler()
cols = first.columns
first = pd.DataFrame(scaler.fit_transform(first), columns = cols)

In [ ]:
first

# FastCluster: Ward

In [ ]:
# tic = time.time()
# Z_ward = flinkage(first, method = 'ward')
# beep = np.sin(2*np.pi*400*np.arange(10000*2)/10000)
# ipd.Audio(beep, rate=10000, autoplay=True)
# toc = time.time()
# print(f"{toc - tic} seconds")
Z_ward = np.loadtxt('/kaggle/input/private-blood-count/Z_ward_mcm2.txt', dtype = 'float')
Z_ward

In [ ]:
# np.savetxt('Z_ward.txt', Z_ward, fmt = "%f")

In [ ]:
# (Z_ward == Z_ward_import).sum(axis = 0)

In [ ]:
len(Z_ward)

In [ ]:
# assume that the clusters are distinct. find an arbitrary cut.
cluster_labels = fcluster(Z_ward, t = 26, criterion = 'maxclust')
cluster_labels = pd.Series(cluster_labels)

In [ ]:
data.Diagnosis[-93:].nunique()

In [ ]:
cluster_labels.value_counts()

In [ ]:
# tic = time.time()
# tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, random_state=42)
# data_2d_tsne = tsne.fit_transform(first)
# beep = np.sin(2*np.pi*400*np.arange(10000*2)/10000)
# ipd.Audio(beep, rate=10000, autoplay=True)
# toc = time.time()

In [ ]:
data_2d_tsne = np.loadtxt('/kaggle/input/private-blood-count/data_2d_tsne_mcm2.txt', dtype = 'float')
data_2d_tsne

In [ ]:
plt.figure(figsize=(20, 20))
plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=1)
plt.title('t-SNE Visualization')
plt.show()

In [ ]:
labelled_points = data.Diagnosis[-93:].copy()
unique_labels = pd.DataFrame(sorted(list(labelled_points.unique())), columns = ['labels'])
unique_labels['frequencies'] = labelled_points.value_counts()[unique_labels.labels].values
unique_labels

In [ ]:
mask = data['Diagnosis'].str.contains('chronic myeloid leukemia chronic phase', case = False)
flag_mask = mask.copy()
flag_mask[flag_mask.isna()] = 0
mask[mask.isna()] = False
mask = mask.astype(bool)
flag_mask[mask] = 1
~mask[-93:]
# ~mask
flag_mask[-93:][~mask[-93:]] = 2
flag_mask
mask
# flag_mask[~flag_mask] = 2
# flag_mask

# subset = data[mask]
# steps: wherever mask is True, replace it with 1
# wherever mask is False and the index is in the last 54 rows (i.e. labelled rows), replace it with 0
# otherwise, replace it with 2. do this using a duplicate mask

In [ ]:
plt.figure(figsize=(20, 20))
plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=1, c=~mask, cmap='viridis')
circle_count_1 = 0
circle_count_2 = mask.sum()
avg_of_points_x = 0
avg_of_points_y = 0
indices_cml_close = []
for i in range(len(data_2d_tsne)):
    if flag_mask[i] == 1:
        if circle_count_1 != 8:
            avg_of_points_x += data_2d_tsne[i, 0]
            avg_of_points_y += data_2d_tsne[i, 1]
            indices_cml_close.append(i)
        circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
        plt.gca().add_patch(circle)
        plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_1), fontsize=12, ha='center', va='center')
        circle_count_1 += 1
    elif flag_mask[i] == 2:
        if circle_count_2 == 51:
            print(i)
        # circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
        # plt.gca().add_patch(circle)
        # plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_2), fontsize=12, ha='center', va='center')
        circle_count_2 += 1
print(data.loc[i, 'Diagnosis'])
avg_of_points_x /= circle_count_1 - 1
avg_of_points_y /= circle_count_1 - 1
print(indices_cml_close)
circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
plt.gca().add_patch(circle)
plt.text(avg_of_points_x + 5, avg_of_points_y + 5, "CML POINTS", fontsize = 25, ha = 'center', va = 'center')
plt.title('t-SNE Visualization')
plt.savefig('t-SNE Visualization v2 mcm2.png')
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=1, c=~mask, cmap='viridis')
circle_count_1 = 0
circle_count_2 = mask.sum()
avg_of_points_x = 0
avg_of_points_y = 0
indices_cml_close = []
for i in range(len(data_2d_tsne)):
    if flag_mask[i] == 1:
        if circle_count_1 != 8:
            avg_of_points_x += data_2d_tsne[i, 0]
            avg_of_points_y += data_2d_tsne[i, 1]
            indices_cml_close.append(i)
            circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
            plt.gca().add_patch(circle)
            plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_1), fontsize=12, ha='center', va='center')
        else:
            print(i)
        circle_count_1 += 1
    elif flag_mask[i] == 2:
        if circle_count_2 == 51:
            print(i, 'hihi')
        # circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
        # plt.gca().add_patch(circle)
        # plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_2), fontsize=12, ha='center', va='center')
        circle_count_2 += 1
print(data.loc[i, 'Diagnosis'])
avg_of_points_x /= circle_count_1 - 1
avg_of_points_y /= circle_count_1 - 1
print(indices_cml_close)
circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
plt.gca().add_patch(circle)
plt.text(avg_of_points_x + 5, avg_of_points_y + 5, "CML POINTS", fontsize = 25, ha = 'center', va = 'center')
plt.title('t-SNE Visualization')
plt.xlim(35, 55)
plt.ylim(-10, 10)
plt.savefig('t-SNE Visualization v2 mcm2.png')
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=1, c=~mask, cmap='viridis')
circle_count_1 = 0
circle_count_2 = mask.sum()
avg_of_points_x = 0
avg_of_points_y = 0
indices_cml_close = []
for i in range(len(data_2d_tsne)):
    if flag_mask[i] == 1:
        if circle_count_1 != 8:
            avg_of_points_x += data_2d_tsne[i, 0]
            avg_of_points_y += data_2d_tsne[i, 1]
            indices_cml_close.append(i)
        else:
            print(i)
        circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
        plt.gca().add_patch(circle)
        plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_1), fontsize=12, ha='center', va='center')
        circle_count_1 += 1
    elif flag_mask[i] == 2:
        if circle_count_2 == 51:
            print(i, 'hihi')
        circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
        plt.gca().add_patch(circle)
        plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_2), fontsize=12, ha='center', va='center')
        circle_count_2 += 1
print(data.loc[i, 'Diagnosis'])
avg_of_points_x /= circle_count_1 - 1
avg_of_points_y /= circle_count_1 - 1
print(indices_cml_close)
circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
plt.gca().add_patch(circle)
plt.text(avg_of_points_x + 5, avg_of_points_y + 5, "CML POINTS", fontsize = 25, ha = 'center', va = 'center')
plt.title('t-SNE Visualization')
plt.savefig('t-SNE Visualization v2 mcm2.png')
plt.show()

In [ ]:
def plot_helper(string, flag_mask, plot_others = True, cluster_labels = cluster_labels, xlim = None, ylim = None, big_plot = True, t = None):
    if big_plot:
        plt.figure(figsize=(20, 20))
        plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=1, c=cluster_labels, cmap='viridis')
        circle_count_1 = 0
        circle_count_2 = mask.sum()
        indices_cml_close = []
        d1 = {}
        d2 = {}
        for i in range(len(data_2d_tsne)):
            if flag_mask[i] == 1:
#                 if circle_count_1 != 8:
#                     avg_of_points_x += data_2d_tsne[i, 0]
#                     avg_of_points_y += data_2d_tsne[i, 1]
#                     indices_cml_close.append(i)
                circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
                plt.gca().add_patch(circle)
                plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.6, str(circle_count_1), fontsize=12, ha='center', va='center')
                d1[circle_count_1] = data.loc[i, 'Diagnosis']
                circle_count_1 += 1
            if plot_others == True:
                if flag_mask[i] == 2:
                    circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
                    plt.gca().add_patch(circle)
                    plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.6, str(circle_count_2), fontsize=12, ha='center', va='center')
                    d2[circle_count_2] = data.loc[i, 'Diagnosis']
                    circle_count_2 += 1
#         avg_of_points_x /= circle_count_1 - 1
#         avg_of_points_y /= circle_count_1 - 1
#         circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
#         plt.gca().add_patch(circle)
#         plt.text(avg_of_points_x - 5, avg_of_points_y + 5, f"{string} POINTS", fontsize = 15, ha = 'center', va = 'center')
        plt.title(f't-SNE Visualization of {string}')
        plt.savefig(f"t-SNE Visualization std iter {string} mcm2.png", bbox_inches = 'tight')
        plt.show()
    
    # Zoomed-in plot if xlim and ylim are provided
    if xlim and ylim:
        plt.figure(figsize = (20, 20))
        plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=6.5, c=cluster_labels, cmap='viridis')
        plt.xlim(xlim)
        plt.ylim(ylim)
        circle_count_1 = 0
        circle_count_2 = mask.sum()
        avg_of_points_x = 0
        avg_of_points_y = 0
        for i in range(len(data_2d_tsne)):
            if flag_mask[i] == 1:
                if circle_count_1 != 8:
                    avg_of_points_x += data_2d_tsne[i, 0]
                    avg_of_points_y += data_2d_tsne[i, 1]
                circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
                plt.gca().add_patch(circle)
                circle_count_1 += 1
            elif flag_mask[i] == 2:
                if plot_others == True:
                    circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
                    plt.gca().add_patch(circle)
                    circle_count_2 += 1
        avg_of_points_x /= circle_count_1 - 1
        avg_of_points_y /= circle_count_1 - 1
        circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
        plt.gca().add_patch(circle)
        if t:
            plt.savefig(f"t-SNE Visualization std iter {t} mcm2.png", bbox_inches = 'tight')
        plt.show()
    if big_plot == True:
        return d1, d2
string = 'Chronic myeloid leukemia chronic phase'
plot_helper(string, flag_mask)
d1, d2 = plot_helper(string, flag_mask, plot_others = True, cluster_labels = cluster_labels, xlim = (30, 50), ylim = (-10, 10))

In [ ]:
def set_mask_generator(string_set):
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        big_mask = np.array([False] * len(data))
        for string in string_set:
#         mask = data['Diagnosis'].str.contains(string, case=False)
            mask = data['Diagnosis'] == string
            flag_mask = mask.copy()
            flag_mask.loc[flag_mask.isna()] = 0  # Use .loc for assignment
            mask.loc[mask.isna()] = False        # Use .loc for assignment
            mask = mask.astype(bool)
            flag_mask[mask] = 1
            flag_mask[-54:][~mask[-54:]] = 2
            big_mask[mask] = True
        plot_helper(string = string, flag_mask = big_mask, plot_others=False, cluster_labels=~big_mask)

In [ ]:
string_set = ['Acute leukemia', 'Acute leukemia monocytic'] #, 'Acute lymphoblastic leukemia', 'Acute myeloid leukemia', 'Acute myeloid leukemia morphologically monocytic', 'Acute myeloid leukemia non M3', 'Acute myelomonocytic leukemia', 'Atypical chronic lymphocytic leukemia', 'Chronic lymphocytic leukemia']

In [ ]:
for string in unique_labels.labels[:8]:
    mask = data['Diagnosis'] == string
    d1, d2 = plot_helper(string, flag_mask = mask, plot_others = False, cluster_labels = ~mask)

In [ ]:
for string in unique_labels.labels[8:16]:
    mask = data['Diagnosis'] == string
    d1, d2 = plot_helper(string, flag_mask = mask, plot_others = False, cluster_labels = ~mask)

In [ ]:
for string in unique_labels.labels[16:]:
    mask = data['Diagnosis'] == string
    d1, d2 = plot_helper(string, flag_mask = mask, plot_others = False, cluster_labels = ~mask)

In [ ]:
unique_labels

In [ ]:
d1

In [ ]:
a = np.array([26, 38, 17, 14, 25, 51, 23, 28, 42, 19, 47, 33, 43, 48, 21])
a.sort()
s2 = pd.Series(d2)
b = np.array(s2.index)
c = np.setdiff1d(b, a)
s2[a].value_counts()

In [ ]:
s2[c].value_counts()

I have 2 ways of approaching this issue:
* I either loop through every single disease and find the mask for each, then plot it accordingly
* I use it point by point

In [ ]:
nunq = data.loc[-54:, 'Diagnosis'].value_counts()

In [ ]:
last_54 = data.iloc[-54:, ].Diagnosis.unique()
last_54.sort()
for l in last_54:
    print(l, nunq[l])

In [ ]:
# def mask_generator(string):
#     mask = data['Diagnosis'].str.contains(string, case = False)
#     flag_mask = mask.copy()
#     flag_mask[flag_mask.isna()] = 0
#     mask[mask.isna()] = False
#     mask = mask.astype(bool)
#     flag_mask[mask] = 1
#     flag_mask[-54:][~mask[-54:]] = 2
#     plot_helper(string, mask, plot_others = False, cluster_labels = ~flag_mask)
# mask_generator('Acute leukemia')

def mask_generator(string):
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
#         mask = data['Diagnosis'].str.contains(string, case=False)
        mask = data['Diagnosis'] == string
        flag_mask = mask.copy()
        flag_mask.loc[flag_mask.isna()] = 0  # Use .loc for assignment
        mask.loc[mask.isna()] = False        # Use .loc for assignment
        mask = mask.astype(bool)
        flag_mask[mask] = 1
        flag_mask[-54:][~mask[-54:]] = 2
    plot_helper(string = string, flag_mask = mask, plot_others=False, cluster_labels=~mask)

In [ ]:
# nunq

In [ ]:
# for n in nunq.index:
#     print(n, nunq[n])
#     mask_generator(n)

In [ ]:
# a = np.array([26, 38, 17, 14, 25, 51, 23, 28, 42, 19, 47, 33, 43, 48, 21, 24, 40])
# a.sort()
# s2 = pd.Series(d2)
# b = np.array(s2.index)
# c = np.setdiff1d(b, a)
# s2[a].value_counts()

In [ ]:
# s2[c].value_counts()

In [ ]:
# cluster_mask = pd.Series([False] * len(first))
# for c in cluster_labels_set:
#     cluster_mask[clusters == c] = True
# plot_helper(~cluster_mask, (30, 50), (-10, 10))
#     # find the indices where the cluster is in the set
#     # set those indices to true, use this to create a new mask.

In [ ]:
# threshold = 198.95
# round(threshold, ndigits = 1)

In [ ]:
# def aux_plot(cluster_labels):
#     plt.figure(figsize=(20, 20))
#     plt.scatter(data_2d_tsne[:, 0], data_2d_tsne[:, 1], s=5, c=cluster_labels, cmap='viridis')
# #     circle_count_1 = 0
# #     circle_count_2 = mask.sum()
# #     avg_of_points_x = 0
# #     avg_of_points_y = 0
# #     indices_cml_close = []
# #     for i in range(len(data_2d_tsne)):
# #         if flag_mask[i] == 1:
# #             if circle_count_1 != 8:
# #                 avg_of_points_x += data_2d_tsne[i, 0]
# #                 avg_of_points_y += data_2d_tsne[i, 1]
# #                 indices_cml_close.append(i)
# #         if cluster_labels[i] == True:
# #             circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='black', fill=False)
# #             plt.gca().add_patch(circle)
# #             plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_1), fontsize=12, ha='center', va='center')
# #                 circle_count_1 += 1
# #         elif flag_mask[i] == 2:
# #             circle = patches.Circle((data_2d_tsne[i, 0], data_2d_tsne[i, 1]), radius=0.1, color='blue', fill=False)
# #             plt.gca().add_patch(circle)
# #             plt.text(data_2d_tsne[i, 0], data_2d_tsne[i, 1] + 0.18, str(circle_count_2), fontsize=12, ha='center', va='center')
# #             circle_count_2 += 1
# #     avg_of_points_x /= circle_count_1 - 1
# #     avg_of_points_y /= circle_count_1 - 1
# #     circle = patches.Circle((avg_of_points_x, avg_of_points_y), radius = 4.6, color = "red", fill = False)
# #     plt.gca().add_patch(circle)
# #     plt.text(avg_of_points_x + 5, avg_of_points_y + 5, "CML POINTS", fontsize = 25, ha = 'center', va = 'center')
#     plt.title('t-SNE Visualization')
#     plt.show()

In [ ]:
# k = 5000
# # Iterate over different threshold levels Z_ward[-1, 2]
# for threshold in np.arange(201.3, 201.5, 0.1):  # or fine-tune based on your dendrogram structure
#     threshold = round(threshold, ndigits = 1)
#     clusters = fcluster(Z_ward, t=threshold, criterion='distance')
#     cluster_labels_set = set(clusters[i] for i in indices_cml_close)
#     cluster_sizes = {label: (clusters == label).sum() for label in cluster_labels_set}  # Dictionary with cluster sizes
#     print(f'{threshold}: {cluster_sizes}')
#     cluster_mask = pd.Series([False] * len(first))
#     for c in cluster_labels_set:
#         cluster_mask[clusters == c] = True
#     aux_plot(~cluster_mask)
#     plot_helper(~cluster_mask, (30, 50), (-10, 10), False, threshold)

In [ ]:
# Z_ward[-1, 2]

In [ ]:
np.savetxt('data_2d_tsne.txt', data_2d_tsne, fmt = "%f")